<a href="https://colab.research.google.com/github/j54854/myColab/blob/main/pom1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 生産管理技術１_6の補助資料
この補助資料では、生産管理技術１の第６回講義で扱った生産計画問題（線形最適化問題）を、PuLP（とフリーのソルバーCBC）を用いて解いてみよう。

## 単一期の生産計画

まず、講義内で取り上げた、単一期の生産計画問題の簡単な数値例を再掲する。

### 決定変数：
$X_1$, $X_2$

### 目的関数：
$f = 4 X_1 +3 X_2$

### 制約条件：
$X_1 +4 X_2 \leq 5200$　　　　(1)

$14 X_1 +4 X_2 \leq 15600$　　　　(2)

$X_1 \leq 1000$　　　　(3)

$X_1, X_2 \leq 0$　　　　(4)

次に、この問題の解決にPuLPを利用できるようにするために、pipでPuLPをインストールしてから、それをインポートする。





In [ ]:
!pip install pulp
import pulp

次に、PuLPで問題（lp1）を定義して、その問題（lp1）に決定変数を登録する。

In [22]:
lp1 = pulp.LpProblem('lp1', sense=pulp.LpMaximize)  # 最小化問題の場合は、sense=pulp.LpMinimizeとする。
X1 = pulp.LpVariable('X1', lowBound=0, upBound=None)  # LowBoud=0を指定することで制約条件（４）を表現できる。
X2 = pulp.LpVariable('X2', lowBound=0, upBound=None) 

次に、問題（lp1）に目的関数を登録する。

In [23]:
lp1 += 4 * X1 +3 *X2

続いて、残りの制約条件(1), (2), (3)を追加していく。

In [24]:
lp1 += X1 +4 *X2 <= 5200
lp1 += 14 *X1 +4 *X2 <= 15600
lp1 += X1 <= 1000

これで問題を登録できた。念のため、登録した問題を確認しておこう。

In [ ]:
print(lp1)

これで準備が整ったので、最後に、ソルバーで解を導出する。lp1.solve()が求解のためのメソッド呼び出しであり、その返り値statusがOptimalになっていれば最適解が得られていることがわかる。

In [ ]:
status = lp1.solve()
print(pulp.LpStatus[status])

最適解とそのときの目的関数値は次のようにして確認すればよい。

In [ ]:
print('Optimal solution: (X1, X2) = ({}, {})'.format(X1.value(), X2.value()))
print('Objective function value: f = {}'.format(lp1.objective.value()))

## 複数期の生産計画

同様に、複数期の生産計画問題も解いておこう。問題自体の再掲は省略するので、講義スライドを確認してほしい。

まず、問題（今度はlp2と命名した）と決定変数を登録しよう。

In [50]:
lp2 = pulp.LpProblem('lp2', sense=pulp.LpMaximize) 

x_keys = [(p, t) for p in (1, 2) for t in (1, 2, 3)]
y_keys = [(p, t) for p in (1, 2) for t in (1, 2)]

x = pulp.LpVariable.dicts('x', x_keys, lowBound=0, upBound=None)
y = pulp.LpVariable.dicts('y', y_keys, lowBound=0, upBound=None)

決定変数は、単一期のときと同じように個別に一つ一つ登録していってもよいが、ここでは、製品番号pと期番号tのペアのタプル(p, t)をキーとした辞書x[(p, t)]として、まとめて登録している。これによって、後の制約条件の登録もfor文を使って効率的に行うことができるようになる。ここでは、詳しい説明は省略するので、もし必要なら、PuLPの公式ドキュメントなどを参照してほしい。

次に目的関数を登録する。なお、pulp.lpDot()は、第1引数を係数ベクトル、第2引き数を変数ベクトルとした内積で定義される線形式を返す関数である。

In [51]:
lp2 += pulp.lpDot((8, 8, 8), (x[(1, 1)], x[(1, 2)], x[(1, 3)])) +pulp.lpDot((6, 6, 6), (x[(2, 1)], x[(2, 2)], x[(2, 3)])) -pulp.lpDot((1, 1), (y[(1, 1)], y[(1, 2)])) -pulp.lpDot((2, 2), (y[(2, 1)], y[(2, 2)]))

続いて、制約条件を登録していこう。

In [52]:
for t in (1, 2, 3):
  lp2 += x[(1, t)] +4 *x[(2, t)] <= 5200
  lp2 += 14 *x[(1, t)] +4 *x[(2, t)] <= 15600

lp2 += x[(1, 1)] -y[(1, 1)] >= 0
lp2 += x[(1, 1)] -y[(1, 1)] <= 500
lp2 += x[(2, 1)] -y[(2, 1)] >= 0
lp2 += x[(2, 1)] -y[(2, 1)] <= 1500
lp2 += y[(1, 1)] +x[(1, 2)] -y[(1, 2)] >= 0
lp2 += y[(1, 1)] +x[(1, 2)] -y[(1, 2)] <= 500
lp2 += y[(2, 1)] +x[(2, 2)] -y[(2, 2)] >= 0
lp2 += y[(2, 1)] +x[(2, 2)] -y[(2, 2)] <= 1200
lp2 += y[(1, 2)] +x[(1, 3)] >= 0
lp2 += y[(1, 2)] +x[(1, 3)] <= 2000
lp2 += y[(2, 2)] +x[(2, 3)] >= 0
lp2 += y[(2, 2)] +x[(2, 3)] <= 900

最後に、解を求めて、結果を確認する。

In [ ]:
print(lp2)
status = lp2.solve()
print(pulp.LpStatus[status])
for p in (1, 2):
  for t in (1, 2, 3):
    print('X_{},{} = {}'.format(p, t, x[(p, t)].value()))
for p in (1, 2):
  for t in (1, 2):
    print('Y_{},{} = {}'.format(p, t, y[(p, t)].value()))
print('Objective function value: f = {}'.format(lp2.objective.value()))

以上のように、pythonのPuLPライブラリ（とそれに付随しているフリーのソルバーCBC）を使うと、簡単に、線形最適化問題の解を求めることができる。